In [28]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelBinarizer, StandardScaler
import os

# Load data
csv_path = os.path.join('..', 'Data', 'Diagested_data', 'Numeric_resume_data.csv')
data = pd.read_csv(csv_path)
df = pd.DataFrame(data)

# Define trait columns
trait_columns = [
    'Leadership', 'Communication', 'Teamwork', 'Problem Solving', 'Creativity',
    'Adaptability', 'Work Ethic', 'Time Management', 'Interpersonal Skills', 
    'Attention to Detail', 'Initiative', 'Analytical Thinking', 'Emotional Intelligence', 
    'Integrity', 'Resilience', 'Cultural Awareness', 'Programming Languages', 
    'Technical Skills', 'Office Tools'
]

# Convert job titles to numerical labels
job_titles = df['Job Title'].unique()
job_domains = df[['Job Title', 'Domain']].drop_duplicates().set_index('Job Title')['Domain'].to_dict()
job_title_to_index = {title: idx for idx, title in enumerate(job_titles)}
index_to_job_title = {idx: title for title, idx in job_title_to_index.items()}
df['Job Label'] = df['Job Title'].map(job_title_to_index)

# Prepare feature and target arrays
X = df[trait_columns].values
y = df['Job Label'].values

# Feature scaling
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Split data into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42)

# Convert labels to one-hot encoding
num_classes = len(job_titles)
lb = LabelBinarizer()
lb.fit(y)  # Fit on the entire set of labels to ensure it has all classes
y_train_onehot = lb.transform(y_train)
y_test_onehot = lb.transform(y_test)

# Define the neural network model
model = Sequential([
    Dense(512, activation='relu', input_shape=(X_train.shape[1],)),
    Dropout(0.5),
    Dense(256, activation='relu'),
    Dropout(0.5),
    Dense(num_classes, activation='softmax')  # Output layer should match the number of unique job titles
])

# Compile the model
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

# Train the model
model.fit(X_train, y_train_onehot, epochs=20, batch_size=64, validation_split=0.2)

# Evaluate the model
loss, accuracy = model.evaluate(X_test, y_test_onehot)
print(f'Test Accuracy: {accuracy:.2f}')

# Function to recommend top 5 jobs with their domains
def recommend_jobs(trait_scores, top_n=5):
    # Predict job probabilities for the given trait scores
    trait_scores = np.array(trait_scores).reshape(1, -1)
    trait_scores_scaled = scaler.transform(trait_scores)  # Scale the input trait scores
    predictions = model.predict(trait_scores_scaled)
    
    # Get the indices of the top N probabilities
    top_indices = np.argsort(predictions[0])[-top_n:][::-1]
    
    # Map indices back to job titles and domains
    top_jobs = [(index_to_job_title[idx], job_domains[index_to_job_title[idx]]) for idx in top_indices]
    return top_jobs

# Example trait scores
trait_scores = [2, 1, 3, 3, 0, 1, 1, 2, 1, 2, 0, 1, 0, 0, 2, 1, 0, 1, 0]

# Recommend top 5 jobs
recommended_jobs = recommend_jobs(trait_scores)
print('Recommended Jobs:')
for job, domain in recommended_jobs:
    print(f'- {job} in the field of {domain}')

# Save the model to a file
model_save_path = os.path.join('..', 'Code', 'Pickles', 'job_recommendation_algorith.h5')
model.save(model_save_path)
print(f'Model saved to {model_save_path}')

Epoch 1/20
17/17 [==============================] - 2s 38ms/step - loss: 7.3700 - accuracy: 0.0000e+00 - val_loss: 7.3808 - val_accuracy: 0.0000e+00
Epoch 2/20
17/17 [==============================] - 0s 22ms/step - loss: 7.2315 - accuracy: 0.0067 - val_loss: 7.4999 - val_accuracy: 0.0038
Epoch 3/20
17/17 [==============================] - 0s 18ms/step - loss: 6.9931 - accuracy: 0.0086 - val_loss: 7.9631 - val_accuracy: 0.0076
Epoch 4/20
17/17 [==============================] - 0s 17ms/step - loss: 6.6264 - accuracy: 0.0095 - val_loss: 8.8111 - val_accuracy: 0.0038
Epoch 5/20
17/17 [==============================] - 0s 18ms/step - loss: 6.2068 - accuracy: 0.0143 - val_loss: 9.3319 - val_accuracy: 0.0038
Epoch 6/20
17/17 [==============================] - 0s 18ms/step - loss: 5.6493 - accuracy: 0.0333 - val_loss: 10.0347 - val_accuracy: 0.0114
Epoch 7/20
17/17 [==============================] - 0s 19ms/step - loss: 4.9638 - accuracy: 0.0829 - val_loss: 10.9592 - val_accuracy: 0.0038
Epo

/home/yair7799/miniconda3/lib/python3.11/site-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


Model saved to ../Code/Pickles/job_recommendation_algorith.h5


/home/yair7799/miniconda3/lib/python3.11/site-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(
